In [ ]:
import os
import sys
import wandb
from huggingface_hub import login

In [ ]:
wandb.login()

In [ ]:
login()

In [7]:
import torch

print(torch.cuda.is_available())

True


In [9]:
!accelerate launch --mixed_precision="bf16" /w/340/vishnouvina/mobilediffusion/lora_distill_training.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1" \
    --dataset_name="fantasyfish/laion-art" \
    --caption_column="text"\
    --resolution=512 --center_crop --random_flip \
    --num_train_epochs=5 \
    --gradient_accumulation_steps=4 \
    --gradient_checkpointing \
    --max_train_steps=10000 \
    --validation_epochs=1 \
    --distill_level="sd_tiny"\
    --prepare_unet="True"\
    --use_peft\
    --use_8bit_adam\
    --max_grad_norm=1\
    --output_weight=0.5\
    --feature_weight=0.5\
    --learning_rate=1e-05\
    --report_to="wandb"\
    --lora_r=4 \
    --lora_alpha=32 \
    --push_to_hub \
    --hub_model_id="sd-laion-art"\
    --output_dir="sd-laion-art"\
    --resume_from_checkpoint="latest"

03/29/2024 02:20:50 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: bf16

{'dynamic_thresholding_ratio', 'clip_sample_range', 'timestep_spacing', 'thresholding', 'variance_type', 'sample_max_value', 'rescale_betas_zero_snr'} was not found in config. Values will be initialized to default values.
{'scaling_factor', 'latents_mean', 'latents_std', 'force_upcast'} was not found in config. Values will be initialized to default values.
{'resnet_out_scale_factor', 'class_embed_type', 'cross_attention_norm', 'projection_class_embeddings_input_dim', 'timestep_post_act', 'addition_embed_type_num_heads', 'dropout', 'reverse_transformer_layers_per_block', 'resnet_time_scale_shift', 'mid_block_type', 'encoder_hid_dim', 'time_embedding_act_fn', 'addition_time_embed_dim', 'addition_embed_type', 'transformer_layers_per_block', 'time_embedding_type', 'conv_out_kernel', 'resnet_skip_time_act', 'conv_in_kernel', '

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipeline = StableDiffusionPipeline.from_pretrained("Vishnou/sd-laion-art", torch_dtype=torch.float16)
prompt = "A man in a suit"
image = pipeline(prompt).images[0]
image.save("my_image.png")


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipeline = StableDiffusionPipeline.from_pretrained("Vishnou/sd-laion-art", torch_dtype=torch.float16)
pipeline.to("cuda")
prompt = "A monkey dancing on a tree"
image = pipeline(prompt).images[0]
image.save("val_imgs_grid.png")

In [ ]:
pipeline.components['unet'].num_parameters()

In [ ]:
pipeline2 = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16)
pipeline2.to("cuda")

pipeline2.components['unet'].num_parameters()